## Feature extraction 

In [1]:
import pandas as pd
X_train = pd.read_csv("/code/data/train.csv")['headline']
y_train = pd.read_csv("/code/data/train.csv")['is_sarcastic']

X_val = pd.read_csv("/code/data/val.csv")['headline']
y_val = pd.read_csv("/code/data/val.csv")['is_sarcastic']

In [2]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer

my_stop_words = text.ENGLISH_STOP_WORDS.union([
    "of", "the", "in", "for", "a", "on", "with", "and", "is",
    "from", "at", "about", "by"
])

vectorizer = TfidfVectorizer(ngram_range=(1,1), stop_words=my_stop_words)

# Initialize the `tfidf_vectorizer` 
#tfidf_vectorizer = TfidfVectorizer(stop_words='english') 
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data 
tfidf_train = tfidf_vectorizer.fit_transform(X_train) 

# Transform the val set 
tfidf_val = tfidf_vectorizer.transform(X_val)

In [3]:
type(tfidf_train)

scipy.sparse.csr.csr_matrix

## Classifiers 

In [4]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

clf1 = MultinomialNB() 

# Predict bag of words

# clf1.fit(tfidf_train, y_train)
# pred1 = clf1.predict(tfidf_val)
# metrics.accuracy_score(y_val, pred1)

In [5]:
from sklearn import svm

clf2 = svm.SVC(gamma='scale', probability=True)

# clf2.fit(tfidf_train, y_train)
# pred2 = clf2.predict(tfidf_val)
# metrics.accuracy_score(y_val, pred2)

In [8]:
from itertools import product
from sklearn.ensemble import VotingClassifier
# Loading some example data

eclf = VotingClassifier(estimators=[('mnb', clf1), ('svm', clf2)],
                        voting='soft', weights=[1, 1])
# clf1 = clf1.fit(X, y)
# clf2 = clf2.fit(X, y)
eclf = eclf.fit(tfidf_train, y_train)
pred3 = eclf.predict(tfidf_val)
metrics.accuracy_score(y_val, pred3)

0.8616352201257862

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB

clf1 = LogisticRegression(multi_class='multinomial', random_state=1,  solver='newton-cg')
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()
clf4 = svm.SVC(gamma='scale', probability=True)
clf5 = MultinomialNB() 


In [17]:
# for i in range(10):
#     print(i)
eclf = VotingClassifier(estimators=[('logr', clf1), ('rfc', clf2), ('gnb', clf3), ('svm', clf4), ('mnb', clf5)],
                        voting='soft', weights=[1,1,1,1,1])
# clf1 = clf1.fit(X, y)
# clf2 = clf2.fit(X, y)
eclf = eclf.fit(tfidf_train.todense(), y_train)


TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [18]:
pred3 = eclf.predict(tfidf_val.todense())
metrics.accuracy_score(y_val, pred3)

0.8022361984626135